# 学习如何进行文本处理
> 对于训练集，自然语言处理过程包括：
> 
>     中文分词、过滤停用词、特征权重计算、特征词选取。
> 
> 而对于测试集：
> 
>     特征权重计算、特征词


## 使用的包 导入

In [1]:
import pandas as pd
import numpy as np
import re
import jieba
import matplotlib.pyplot as plt


## 原始数据及基本信息

In [2]:
data = pd.read_csv('Data/完整数据_暴雨_交通.csv')
data.head()

,id,bid,user_id,用户昵称,微博正文,头条文章url,发布位置,艾特用户,话题,转发数,评论数,点赞数,发布时间,发布工具,微博图片url,微博视频url,retweet_id
0,4632157211069133,Kduyt4u85,7073634525,中国交通,#我家门口那条路#5月1日20时至2日20时，新疆天山西部、西藏中部等地部分地方有雨夹雪。西...,NaN,NaN,NaN,"我家门口那条路,国家综合立体交通网规划纲要,中国路网",1,3,6,2021-05-01 21:30,微博 weibo.com,['https://wx3.sinaimg.cn/large/007IIftrly1gq34...,NaN,NaN
1,4632138676964878,Kdu4ztdSK,6360489687,昌江交警,"#五一我在岗##畅行中国,交警同行#五一，大雨滂沱，而你无阻。你是烈日暴雨下的指示牌，是人民...",NaN,NaN,"海南交警,中国交通频道,公安部交通管理局","五一我在岗,畅行中国,交警同行",0,0,0,2021-05-01 20:16,新版微博 weibo.com,['https://wx2.sinaimg.cn/large/006WrXVlgy1gq36...,NaN,NaN
2,4632091960806995,KdsRe3nW3,2098013967,高州天气,【高州天气预报】今晚到明天白天，多云，有（雷）阵雨局部大雨，气温22-29℃，相对湿度65%...,NaN,NaN,NaN,NaN,0,0,0,2021-05-01 17:10,微博 weibo.com,NaN,NaN,NaN
3,4632061941647124,Kds4O6Uuw,5323286636,广德公安在线,#我在岗位上#【狂风暴雨中交警保畅通】4月30日晚，狂风暴雨突然袭来。@广德交警面对恶劣天气...,NaN,NaN,广德交警,"我在岗位上,我为群众办实事",0,0,1,2021-05-01 15:11,360安全浏览器,['https://wx1.sinaimg.cn/large/005OfY1Sly1gq2y...,NaN,NaN
4,4632029499493949,KdretDPOd,2578964252,广德交警,#我为群众办实事#五一前夕，暴雨来袭！广德公安交警在岗在位，加强队所联勤，全力疏导交通、清理...,NaN,NaN,"公安部交通管理局,安徽公安交警在线,广德公安在线,宣城公安交警在线",我为群众办实事,0,0,0,2021-05-01 13:02,荣耀20 PRO,['https://wx2.sinaimg.cn/large/99b7df1cly1gq2u...,NaN,NaN


In [3]:
data.shape

(15155, 17)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15155 entries, 0 to 15154
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          15155 non-null  int64  
 1   bid         15155 non-null  object 
 2   user_id     15155 non-null  int64  
 3   用户昵称        15155 non-null  object 
 4   微博正文        15155 non-null  object 
 5   头条文章url     21 non-null     object 
 6   发布位置        1127 non-null   object 
 7   艾特用户        4092 non-null   object 
 8   话题          10761 non-null  object 
 9   转发数         15155 non-null  int64  
 10  评论数         15155 non-null  int64  
 11  点赞数         15155 non-null  int64  
 12  发布时间        15155 non-null  object 
 13  发布工具        14913 non-null  object 
 14  微博图片url     6400 non-null   object 
 15  微博视频url     5130 non-null   object 
 16  retweet_id  0 non-null      float64
dtypes: float64(1), int64(5), object(11)
memory usage: 2.0+ MB


In [5]:
data.isnull().sum()

id                0
bid               0
user_id           0
用户昵称              0
微博正文              0
头条文章url       15134
发布位置          14028
艾特用户          11063
话题             4394
转发数               0
评论数               0
点赞数               0
发布时间              0
发布工具            242
微博图片url        8755
微博视频url       10025
retweet_id    15155
dtype: int64

## 随机的200行，测试代码

In [6]:
df = data.sample(n=200, replace=False, random_state=1)
# df.reset_index(drop=True, inplace=True)
df.shape

(200, 17)

In [8]:
df['微博正文']

6826     【面对暴雨，请收好这份#避险生存指南#】#蜀黍带你涨姿势#近日来，河南两小时暴雨记录又被刷新...
1432     #河南高速路况#截至2021年9月9日,19:00，目前省内高速通行情况：1、因收费站临时抢...
6965     郑州大暴雨灾害这波，今天回来又看到一交通事故，一个大叔满身血淋淋坐在车前，前几日杭州电瓶车骑...
12476    【暖心！#交警暴雨中执勤频频被路人送伞#】6月10日17时许，浙江温州泰顺县一路口严重拥堵，...
3023     【湖南多地迎暴雨！这些收费站仍在管制中】今日晚间，湖南部分地区突降暴雨，905中国交通广播派...
                               ...                        
3014     暴雨来袭，行人户外活动谨防雷电！注意安全！转发，提醒更多人↓↓↓#长沙打雷##交通安全不放假...
6469     #河南高速路况#截至2021年7月29日11:05，1、因积水，禁止上下站的收费站有：菏宝高...
13372    #大众观天下#【暖心！#交警暴雨中执勤频频被路人送伞#】6月10日17时许，浙江温州泰顺县一...
11309    珠海交通【突发：受暴雨影响，梅界路交蓝盾路路口暂时禁止通行！】受暴雨影响，梅界路交蓝盾路奥园...
5623     【#外卖小哥雷电暴雨中指挥交通#：能帮大家尽快回家挺开心】据@看看新闻KNEWS消息：近日，...
Name: 微博正文, Length: 200, dtype: object

### 1 文本去噪

In [10]:
def text_clean(text):
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", "", text)  # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r"\[\S+\]", "", text)     # 去除表情符号
    # text = re.sub(r"#\S+#", "", text)     # 去除话题内容
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)      # 去除网址
    
    EMAIL_REGEX = re.compile(r"[-a-z0-9_.]+@(?:[-a-z0-9]+\.)+[a-z]{2,6}", re.IGNORECASE)
    text = re.sub(EMAIL_REGEX, "", text)    # 去除邮件 
    
    text = text.replace("转发微博", "")      # 去除无意义的词语
    text = text.replace("网页链接", "")
    text = re.sub(r"\s+", " ", text)        # 合并正文中过多的空格
    return text.strip()


In [40]:
print(text_clean("太过分了@Rexzhenghao //@Janie_Zhang:招行最近负面新闻越来越多呀..."))
print(text_clean("希望你?得好?我本＂?@Pete三姑父\xa0肥血?史＂[晕][哈哈]@Pete三姑父"))
print(text_clean("回复@钱旭明QXM:[嘻嘻][嘻嘻] //@钱旭明QXM:杨大哥[good][good]"))
print(text_clean("【#赵薇#：正筹备下一部电影 但不是青春片....http://t.cn/8FLopdQ"))
print(text_clean("[酷]//@芊如_GZ:[哈哈]// @布丁clout : 哈哈 // @audrey-panda :试了我家那位，说我属于＂放心的没法看的＂[晕] // @芊如_GZ :[晕]"))

太过分了招行最近负面新闻越来越多呀...
希望你?得好?我本＂?@Pete三姑父 肥血?史＂
杨大哥
【#赵薇#：正筹备下一部电影 但不是青春片....
哈哈 试了我家那位，说我属于＂放心的没法看的＂


In [16]:
tweets = df.loc[:, ['微博正文']]
tmp = tweets.iloc[6, 0]
tmp

'【雨中，你们坚守的样子真帅！】6月17日早上8点，正值出行早高峰，一场突袭的暴雨倾注了这座城市，为确保雨天辖区道路安全畅通，避免因大雨造成的交通事故，@南充交警一大队\xa0全体民辅警，冒雨在辖区各个主要路段执勤，有的甚至来不及换上雨衣，淋着雨指挥车辆，疏导交通，全力保障恶劣天气下市民出行畅通，成为大雨中最美丽的风景线。@四川公安@四川交警@四川长安网L南充公安的微博视频'

In [22]:
res1 = text_clean(tmp)
res1

'【雨中，你们坚守的样子真帅！】6月17日早上8点，正值出行早高峰，一场突袭的暴雨倾注了这座城市，为确保雨天辖区道路安全畅通，避免因大雨造成的交通事故，@南充交警一大队 全体民辅警，冒雨在辖区各个主要路段执勤，有的甚至来不及换上雨衣，淋着雨指挥车辆，疏导交通，全力保障恶劣天气下市民出行畅通，成为大雨中最美丽的风景线。'

### 2 中文分词

In [29]:

# jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str, use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut("我来到北京清华大学【】", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学【】", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Paddle enabled successfully......


Paddle Mode: 我/来到/北京清华大学
Paddle Mode: 乒乓球/拍卖/完/了
Paddle Mode: 中国科学技术大学
Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学/ 【】
Default Mode: 我/ 来到/ 北京/ 清华大学/ 【/ 】
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [34]:
# jieba.lcut() 直接放回列表
seg_list = jieba.lcut("我来到北京清华大学【】", cut_all=True)
# print("Full Mode: " + "/ ".join(seg_list))  # 全模式
print(seg_list)

['我', '来到', '北京', '清华', '清华大学', '华大', '大学', '【】']


In [36]:
# 载入词典
# 用法： jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径
# 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
# 词频省略时使用自动计算的能保证分出该词的词频。

In [38]:
text = '疯狂囤金，中国大妈震惊华尔街'
jieba.load_userdict('CiDian.txt')
wordlist = jieba.lcut(text)
wordlist


['疯狂', '囤金', '，', '中国大妈', '震惊', '华尔街']